### Autonomous Driving in Airsim: Testing the driving

In [ ]:
from keras.models import load_model
import sys
import numpy as np
import os
import airsim
from AirSimClient import *

MODEL_PATH = "model/models/model_ls_final.h5"

    
os.chdir('C:/Users/Pascal/AppData/Local/Programs/Python/Python35/Scripts/AppliedDataScience/AirSim_Experiments/')


In [ ]:
model = load_model(MODEL_PATH)

# connect to the AirSim simulator 
client = airsim.CarClient()
client.confirmConnection()
client.enableApiControl(True)
car_controls = airsim.CarControls()

car_controls.steering = 0
car_controls.throttle = 0
car_controls.brake = 0


image_in = np.zeros((1, 59, 255, 3))

# function to request an image from the front camera
def get_image():
    image_response = client.simGetImages([ImageRequest(0, AirSimImageType.Scene, False, False)])[0]
    image1d = np.fromstring(image_response.image_data_uint8, dtype=np.uint8)
    image_rgba = image1d.reshape(image_response.height, image_response.width, 4)
    
    return image_rgba[76:135,0:255,0:3].astype(float)

# start driving
while (True):
    car_state = client.getCarState()
    
    # accelerate if below 8 m/s (~30 km/h)
    if (car_state.speed < 8):
        car_controls.throttle = 1.0
    else:
        car_controls.throttle = 0.0
    
    image_in[0] = get_image()
    
    # predict the steering with the CNN
    model_output = model.predict(image_in)
    car_controls.steering = float(model_output[0][0])
    
    print('Sending steering = {0}, throttle = {1}'.format(car_controls.steering, car_controls.throttle))
    
    client.setCarControls(car_controls)
  